In [62]:
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [63]:
tokenizer = get_tokenizer('basic_english')
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

In [102]:
def return_iters():
    from ast import literal_eval as make_tuple
    global db
    train_iter=[]
    test_iter=[]
    file = open(db,'r')
    # file2 = open("PoliticalCompassML/databaseTop.txt",'r')
    mapping = {
    "Libertarian Left":1,
    "Libertarian Right":2,
    "Authoritarian Left":3,
    "Authoritarian Right":4,
    }
    for line in file:
        opinion, text = make_tuple(line)
        train_iter+=[(mapping[opinion],text)]
        test_iter+=[(mapping[opinion],text)]
    # for line in file2:
    #     opinion, text = make_tuple(line)
    #     train_iter+=[(mapping[opinion],text)]
    #     test_iter+=[(mapping[opinion],text)]
    train_iter=iter(train_iter)
    test_iter=iter(test_iter)
    file.close()
    # file2.close()
    return train_iter,test_iter

In [103]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1
train_iter,test_iter= return_iters()
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])


In [104]:
import torch
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [105]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [106]:
train_iter,test_iter= return_iters()

dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [107]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        # self.fc = nn.Linear(embed_dim, num_class)
        # self.fc = nn.SELU()
        self.fc = nn.ReLU()
        # self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [108]:
train_iter,test_iter= return_iters()

num_class = len(set([label for (label, text) in train_iter]))
# print(train_iter)
# print([label for (label, text) in train_iter])
print(num_class)
vocab_size = len(vocab)
emsize = 128
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

4


In [109]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 50
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [112]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
import tensorflow as tf
# Hyperparameters
global db
db = "Merged_No_Dupes_DB"
EPOCHS = 20 # epoch
LR = 0.15 # learning rate
BATCH_SIZE = 32 # batch size for training
run_ledger= open("Run_Ledger.txt",'a')
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
# optimizer = torch.optim.Adam(model.parameters(),lr=0.005,betas=(0.9,0.999),eps=1e-08,weight_decay=0,amsgrad=False)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter,test_iter= return_iters()

train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)
first_flag = True
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    if first_flag:
        run_ledger.write("Database file: "+db+"\t"+"Epochs:" +str(EPOCHS)+"\t"+"LR: "+str(LR)+"\t"+"Batch Size: "+str(BATCH_SIZE)+"\tinit accu_val:"+str(accu_val)+"\n")
        first_flag=False
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)
run_ledger.write("Final accu:\t"+str(accu_val)+"\n\n")
accu_test = evaluate(test_dataloader)
out = 'test accuracy {:8.3f}'.format(accu_test)
function="relu"
print(out)
run_ledger.write(out+"function:"+function+'\n')
run_ledger.close()

| epoch   1 |    50/  644 batches | accuracy    0.359
| epoch   1 |   100/  644 batches | accuracy    0.360
| epoch   1 |   150/  644 batches | accuracy    0.361
| epoch   1 |   200/  644 batches | accuracy    0.335
| epoch   1 |   250/  644 batches | accuracy    0.331
| epoch   1 |   300/  644 batches | accuracy    0.339
| epoch   1 |   350/  644 batches | accuracy    0.361
| epoch   1 |   400/  644 batches | accuracy    0.346
| epoch   1 |   450/  644 batches | accuracy    0.347
| epoch   1 |   500/  644 batches | accuracy    0.354
| epoch   1 |   550/  644 batches | accuracy    0.338
| epoch   1 |   600/  644 batches | accuracy    0.354
-----------------------------------------------------------
| end of epoch   1 | time:  2.94s | valid accuracy    0.353 
-----------------------------------------------------------
| epoch   2 |    50/  644 batches | accuracy    0.332
| epoch   2 |   100/  644 batches | accuracy    0.361
| epoch   2 |   150/  644 batches | accuracy    0.341
| epoch  

In [92]:
mapping = {
1:"Libertarian Left",
2:"Libertarian Right",
3:"Authoritarian Left",
4:"Authoritarian Right",
}
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

model = model.to("cpu")
ex_text_str = """
The rise of the two income household is one of the big reasons nobody likes to talk about. Once it became normal for women to be in the workforce while also raising kids, housing prices reflected the new higher median household income. Two incomes used to be a luxury and now it’s a necessity simply due to the laws of supply and demand.

This isn’t the only reason houses are so expensive, but it’s a big reason that people don’t like to talk about.
"""
# https://old.reddit.com/r/PoliticalCompassMemes/comments/x774os/conservative_you_say_sounds_fine_to_me/inbbz52/
print("This is a %s comment" %mapping[predict(ex_text_str, text_pipeline)])

This is a Libertarian Right comment
